In [71]:
# Connect to Flask API

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service

# Check the selenium version insalled
import selenium
print(selenium.__version__)

WEB_DRIVER_LOCATION = "/app/geckodriver"
TIMEOUT = 5

#firefox_options = FirefoxOptions()
# If you comment the following line, a browser will show ...
#firefox_options.add_argument("--headless")

# Adding a specific user agent
#firefox_options.add_argument("user-agent=fri-ieps-TEST")

#service = Service(executable_path=WEB_DRIVER_LOCATION)
#driver = webdriver.Firefox(service=service, options=firefox_options)
#driver.get("https://vreme.arso.gov.si")
#time.sleep(TIMEOUT)

#html = driver.page_source

# print(f"Retrieved Web content (truncated to first 900 chars): \n\n'\n{html[:900]}\n'\n")

import requests
from requests.adapters import HTTPAdapter

session = requests.Session()
session.mount('http://', HTTPAdapter(max_retries=1))
session.mount('https://', HTTPAdapter(max_retries=2))

AUTH = ("admin", "admin") # Development authentication
ENDPOINT = "https://api:5000" # Accessible inside Docker network

response = requests.get(ENDPOINT + "/", verify = False, auth = AUTH)

if response.status_code == 200:
    print(response.json()['message'])
else:
    print(response.json()['message'])


4.18.1
Connected to database!


/opt/conda/envs/wier/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [81]:
class Site:
    def __init__(self, domain, robots_content, sitemap_content, crawl_delay):
        self.domain = domain
        self.robots_content = robots_content
        self.sitemap_content = sitemap_content
        self.crawl_dielay = crawl_delay
        
class Page:
    def __init__(self, url, domain, page_type_code=None, content_hash=None, http_status_code=None, accessed_time=None, data_type_code=None, html_content=None):
        self.page_type_code = page_type_code
        self.url = url
        self.html_content = html_content
        self.content_hash = content_hash
        self.http_status_code = http_status_code
        self.accessed_time = accessed_time
        self.domain = domain
        self.data_type_code = data_type_code
        
class Image:
    def __init__(self, filename, content_type, data, accessed_time):
        self.filename = filename
        self.content_type = content_type
        self.data = data
        self.accessed_time = accessed_time

In [82]:
import urllib
from urllib.parse import urlparse, urlunparse
from bs4 import BeautifulSoup
import threading
from queue import Queue
import urllib.robotparser
import datetime

class Crawler:
    def __init__(self, initial_seed, num_workers):
        self.initial_seed = initial_seed
        self.frontier = Queue()
        # self.visited = set()
        self.num_workers = num_workers
        self.web_driver = self.initialize_web_driver()
        # self.lock = threading.Lock()
    
    def initialize_web_driver(self):
        options = FirefoxOptions()
        options.add_argument("--headless")
        options.add_argument("user-agent=fri-wier-oskapha")
        service = Service(executable_path=WEB_DRIVER_LOCATION)
        driver = webdriver.Firefox(service=service, options=options)
        driver.set_page_load_timeout(10)
        return driver
        
    def canonicalize_url(self, url):
        """
        Canonicalizes a URL by removing redundant parts and ensuring consistency.
        """
        parsed_url = urlparse(url)
        parsed_url = parsed_url._replace(fragment='')
        parsed_url = parsed_url._replace(scheme=parsed_url.scheme.lower(), netloc=parsed_url.netloc.lower())
        parsed_url = parsed_url._replace(path=urlparse(parsed_url.geturl()).path)
        
        if parsed_url.path and not parsed_url.path.endswith('/'):
            parsed_url = parsed_url._replace(path=parsed_url.path + '/')
        
        canonicalized_url = urlunparse(parsed_url)
        
        return canonicalized_url
    
    def robots_allow(self, url):
        domain = self.get_domain(url)
        robot_parser = urllib.robotparser.RobotFileParser()
        robot_parser.set_url(f'https://{domain}/robots.txt')
        robot_parser.read()
        return robot_parser.can_fetch(GROUP_NAME, url)
    
    def check_duplicates(self, page):
        # Check if the page is already in the database by checking the content hash
        content_hashes = []
        # Get function to get the content hash of all pages in the database
        
        for content_hash in content_hashes:
            if content_hash == page.content_hash:
                return True
            else:
                return False
    
    def get_domain(self, url):
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        if "www." in domain:
            domain = domain.replace("www.", "")
            
        return domain
        
    def set_frontier(self):
        print('Setting frontier')

        # Search the database
        # Pages with PAGE_TYPE = FRONTIER
        
        # self.frontier.put(page)
                
        print('Frontier size: %s', self.frontier.qsize())
        
    def initialize_seed(self):
        print('Initializing seed')
        # Add URLs to frontier passed from the argument
        # In case there are no URLs passed from the argument, search for the seed URLs in the database
        if len(self.initial_seed) != 0:
            for url in self.initial_seed:
                #  Insert page into the frontier to DB
                page = Page(url, self.get_domain(url))
                
                # Insert each page into frontier
                
                self.frontier.put(page)
                
        print('Frontier size: %s', self.frontier.qsize())
        
    def get_links(page):
        print('Getting links')
        links = []
            
        return links
                
    def crawl_webpage(self, page):
        print('Started crawling', page.url)
        
        try:
            # Implement robots.txt
            
            # Read the page
            
            response = requests.head(page.url, allow_redirects=True, timeout=10)
            page.http_status_code = response.status_code
            page.accessed_time = datetime.datetime.now()
            page.content_type = response.headers['Content-Type']
            
            # Is it HTML
            if "text/html" in response.headers['Content-Type']:
                page.page_type_code = "HTML"
                self.web_driver.get("http://" + page.domain)
                page.html_content = self.web_driver.page_source
                
                
                # Check duplicates
                
                
            else:
                # Else it is a binary file
                print("It's a binary file!")
                
                page.html_content = None
                page.page_type_code = "BINARY"
                if "application/pdf" == page.content_type:
                    page.data_type = "PDF"
                    
                elif "application/msword" == page.content_type:
                    page.data_type = "DOC"
                
                elif "application/vnd.openxmlformats-officedocument.wordprocessingml.document" == page.content_type:
                    page.data_type = "DOCX"
                    
                elif "application/vnd.ms-powerpoint" == page.content_type:
                    page.data_type = "PPT"
                    
                elif "application/vnd.openxmlformats-officedocument.presentationml.presentation" == page.content_type:
                    page.data_type = "PPTX"
            
            # If everything went fine update database page entry
            print('Crawled %s', response, page.content_type)
        except Exception as e:
            # Update database page entry with PAGE_TYPE = TIMEOUT
            print('Error crawling %s: %s', page, e)
            return

In [75]:
# Start of crawler

import urllib3
import requests
from requests.adapters import HTTPAdapter
import warnings

INITIAL_SEED = [
    'https://www.gov.si/',
    'https://spot.gov.si',
    'https://e-uprava.gov.si',
    'https://www.e-prostor.gov.si'
]

NUMBER_OF_WORKERS = 1

GROUP_NAME='wier2024'

In [79]:
crawler = Crawler(initial_seed=INITIAL_SEED, num_workers=NUMBER_OF_WORKERS)
crawler.initialize_seed()

Initializing seed
Frontier size: %s 4


In [80]:
while crawler.frontier.qsize() > 0:
    # Dequeue element
    element = crawler.frontier.get()
    
    # Process the element
    crawler.crawl_webpage(page=element)
    
    crawler.frontier.task_done()

Started crawling https://www.gov.si/
Crawled %s <Response [200]> text/html; charset=utf-8
Started crawling https://spot.gov.si
Crawled %s <Response [200]> text/html; charset=utf-8
Started crawling https://e-uprava.gov.si
Error crawling %s: %s <__main__.Page object at 0x7f8d38dc6190> 'content-type'
Started crawling https://www.e-prostor.gov.si
Crawled %s <Response [200]> text/html; charset=utf-8
